<a href="https://colab.research.google.com/github/teamgaon/KLUE/blob/main/220215_sm_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT finetuning

## 특허문서의 title, abstract, claim과 산업분류 코드를 활용하여 IPC를 예측

In [1]:
# !pip install gluonnlp
# !pip install mxnet
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# # !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
# !pip install transformers
# !pip install sentencepiece
# !pip install kobert-transformers
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 344 kB 5.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595736 sha256=0e97bd2ea0945e0f44d755ecad76649e98ae4912ecaf8f0cd580df60c1216c92
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 47.3 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-hatd7x3b/kobert-tokenizer_42c4befec0534657b0bfba863062ad83
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-hatd7x3b/kobert-tokenizer_42c4befec0534657b0bfba863062ad83
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=ee05e5034b032d516adee66668de11fb52305f9c0594

In [1]:
!pip install gluonnlp
!pip install mxnet
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-xqm58j5_
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-xqm58j5_


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
# from kobert.utils import get_tokenizer
# from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
# from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split
from kobert_transformers import get_tokenizer
from kobert_transformers import get_kobert_model, get_distilkobert_model

In [2]:
from kobert import get_pytorch_kobert_model

In [56]:
# from kobert_transformers import get_tokenizer
# tokenizer = get_tokenizer()

In [4]:
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

# model = AutoModel.from_pretrained("klue/roberta-large")
# # tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

MODEL_NAME = 'klue/roberta-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config = AutoConfig.from_pretrained(MODEL_NAME)
config.num_labels = 3

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config)

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classif

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df = pd.read_csv('/content/drive/MyDrive/KLUE/train_data.csv')

In [7]:
df

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral
...,...,...,...,...
24993,24993,"오페라에 비하여 오라토리오에서는 독창보다도 합창이 중시되며, 테스토 또는 이스토리쿠...",오라토리오에서 테스토의 역할이 가장 중요하다.,neutral
24994,24994,지하철역까지 걸어서 5분 정도 걸립니다.,지하철역까지 도보로 5분 정도 걸립니다.,entailment
24995,24995,한편 이날 중앙방역대책본부는 집단 감염이 발생한 음식점 관련 역학조사 결과를 공개했다.,중악방역대책본부는 집단 감염과 관련한 모든 정보를 비공개했다.,contradiction
24996,24996,마미손이 랩을 하자 시청자들은 그의 정체를 파악했다.,시청자들은 마미손의 정체를 안다.,entailment


In [8]:
df['text'] = '[CLS]' + df['premise'] + '[SEP] ' + df['hypothesis'] + '[SEP]'

In [9]:
df

,index,premise,hypothesis,label,text
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction,"[CLS]씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나..."
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction,[CLS]삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 ...
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment,[CLS]이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.[SEP] 예측적 ...
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral,[CLS]광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민...
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral,"[CLS]진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 ..."
...,...,...,...,...,...
24993,24993,"오페라에 비하여 오라토리오에서는 독창보다도 합창이 중시되며, 테스토 또는 이스토리쿠...",오라토리오에서 테스토의 역할이 가장 중요하다.,neutral,"[CLS]오페라에 비하여 오라토리오에서는 독창보다도 합창이 중시되며, 테스토 또는 ..."
24994,24994,지하철역까지 걸어서 5분 정도 걸립니다.,지하철역까지 도보로 5분 정도 걸립니다.,entailment,[CLS]지하철역까지 걸어서 5분 정도 걸립니다.[SEP] 지하철역까지 도보로 5분...
24995,24995,한편 이날 중앙방역대책본부는 집단 감염이 발생한 음식점 관련 역학조사 결과를 공개했다.,중악방역대책본부는 집단 감염과 관련한 모든 정보를 비공개했다.,contradiction,[CLS]한편 이날 중앙방역대책본부는 집단 감염이 발생한 음식점 관련 역학조사 결과...
24996,24996,마미손이 랩을 하자 시청자들은 그의 정체를 파악했다.,시청자들은 마미손의 정체를 안다.,entailment,[CLS]마미손이 랩을 하자 시청자들은 그의 정체를 파악했다.[SEP] 시청자들은 ...


In [10]:
my_dict={}
for i in range(len(df['label'].unique())):
  my_dict[df['label'].unique()[i]] = i

def label_to_num(text:str):
  return my_dict[text]

df['target'] = df['label'].map(label_to_num)

In [11]:
df

,index,premise,hypothesis,label,text,target
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction,"[CLS]씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나...",0
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction,[CLS]삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 ...,0
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment,[CLS]이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.[SEP] 예측적 ...,1
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral,[CLS]광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민...,2
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral,"[CLS]진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 ...",2
...,...,...,...,...,...,...
24993,24993,"오페라에 비하여 오라토리오에서는 독창보다도 합창이 중시되며, 테스토 또는 이스토리쿠...",오라토리오에서 테스토의 역할이 가장 중요하다.,neutral,"[CLS]오페라에 비하여 오라토리오에서는 독창보다도 합창이 중시되며, 테스토 또는 ...",2
24994,24994,지하철역까지 걸어서 5분 정도 걸립니다.,지하철역까지 도보로 5분 정도 걸립니다.,entailment,[CLS]지하철역까지 걸어서 5분 정도 걸립니다.[SEP] 지하철역까지 도보로 5분...,1
24995,24995,한편 이날 중앙방역대책본부는 집단 감염이 발생한 음식점 관련 역학조사 결과를 공개했다.,중악방역대책본부는 집단 감염과 관련한 모든 정보를 비공개했다.,contradiction,[CLS]한편 이날 중앙방역대책본부는 집단 감염이 발생한 음식점 관련 역학조사 결과...,0
24996,24996,마미손이 랩을 하자 시청자들은 그의 정체를 파악했다.,시청자들은 마미손의 정체를 안다.,entailment,[CLS]마미손이 랩을 하자 시청자들은 그의 정체를 파악했다.[SEP] 시청자들은 ...,1


In [12]:
my_dict

{'contradiction': 0, 'entailment': 1, 'neutral': 2}

In [13]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [14]:
def remove_enter(text:str):
  return text.replace('\n',"")

df['text'] = df['text'].map(remove_enter)

In [15]:
df['text'].loc[0]

'[CLS]씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이 넓고 평평한 백사장이나 마당에서 모여 서로 힘과 슬기를 겨루는 것이다.[SEP] 씨름의 여자들의 놀이이다.[SEP]'

In [16]:
df['text'] = df['text'].astype('str')

In [17]:
# dataset_train = df[['No.', 'text', 'target_label']][:int(len(df)*0.8)]
# dataset_test = df[['No.', 'text', 'target_label']][int(len(df)*0.8):]
dataset_train, dataset_test = train_test_split(df[['index', 'text', 'target']], test_size=0.2, random_state=42, shuffle=True)
dataset_train.to_csv('dataset_train.txt', index=False, header=True, sep='\t', encoding='utf8')
dataset_test.to_csv('dataset_test.txt', index=False, header=True, sep='\t', encoding='utf8')

In [18]:
dataset_train = nlp.data.TSVDataset('/content/dataset_train.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/dataset_test.txt', field_indices=[1,2], num_discard_samples=1)

In [19]:
dataset_train[4]

['[CLS]코즈웨이베이역이랑 5분거리인것도 좋앗습니다[SEP] 5분거리에 코즈웨이베이역이 있습니다.[SEP]', '1']

In [25]:
from gluonnlp.data import SentencepieceTokenizer
from kobert import get_tokenizer
tok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [26]:
from kobert_transformers import get_tokenizer
# tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(sp, vocab, lower=False)

In [30]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [31]:
## Setting parameters
max_len = 256
batch_size = 8
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [32]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

TypeError: ignored

In [108]:
next(iter(data_train))

(array([   2,  702,  638,  315,  517,  363,  807, 7119, 7926, 1259, 7234,
        1106, 6079, 3803, 7784, 1767, 4012, 6003, 6412, 5859, 7095, 3394,
        2856, 7095, 1713, 2095, 7344, 7078, 1442, 5330, 5400, 1771,  517,
          54,  702,  687,  282,  333,  517,  363,  807, 7119, 7926, 1259,
        7234, 1106, 3803, 5965, 6234, 6896, 2095, 7344, 7078, 1442, 5338,
         517,   54,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [109]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [110]:
len(df['target'].unique())

3

In [111]:
# class BERTClassifier(nn.Module):
#     def __init__(self,
#                  bert,
#                  hidden_size = 768,
#                  num_classes=520,
#                  dr_rate=None,
#                  params=None):
#         super(BERTClassifier, self).__init__()
#         self.bert = bert
#         self.dr_rate = dr_rate
                 
#         self.classifier = nn.Linear(hidden_size , num_classes)
#         if dr_rate:
#             self.dropout = nn.Dropout(p=dr_rate)
    
#     def gen_attention_mask(self, token_ids, valid_length):
#         attention_mask = torch.zeros_like(token_ids)
#         for i, v in enumerate(valid_length):
#             attention_mask[i][:v] = 1
#         return attention_mask.float()

#     def forward(self, token_ids, valid_length, segment_ids):
#         attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
#         _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
#         if self.dr_rate:
#             out = self.dropout(pooler)
#         return self.classifier(out)

In [112]:
# model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [113]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [114]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [115]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [116]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [117]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [118]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # torch.cuda.empty_cache()

  0%|          | 0/2500 [00:00<?, ?it/s]

ValueError: ignored

In [ ]:
print(test_acc / (batch_id+1))